In [29]:
import pandas as pd
import numpy as np

import scipy.sparse as sparse

from implicit.als import AlternatingLeastSquares

In [4]:
raw_data = pd.read_table('usersha1-artmbid-artname-plays.tsv')

In [6]:
raw_data = raw_data.drop(raw_data.columns[1], axis=1)
raw_data.columns = ['user', 'artist', 'plays']

In [7]:
raw_data.head()

,user,artist,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
1,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
2,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
3,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691


In [8]:
data = raw_data.dropna()

In [10]:
data['user_id'] = data['user'].astype("category").cat.codes
data['artist_id'] = data['artist'].astype("category").cat.codes

/Users/aleksandrkondraskin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/aleksandrkondraskin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
data.head()

,user,artist,plays,user_id,artist_id
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,0,90933
1,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897,0,185367
2,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717,0,106704
3,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706,0,155241
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691,0,220128


In [12]:
item_lookup = data[['artist_id', 'artist']].drop_duplicates()
item_lookup['artist_id'] = item_lookup.artist_id.astype(str)

In [13]:
item_lookup.head()

,artist_id,artist
0,90933,die Ärzte
1,185367,melissa etheridge
2,106704,elvenking
3,155241,juliette & the licks
4,220128,red hot chili peppers


In [14]:
data = data.drop(['user', 'artist'], axis=1)

In [15]:
data.head()

,plays,user_id,artist_id
0,1099,0,90933
1,897,0,185367
2,717,0,106704
3,706,0,155241
4,691,0,220128


In [16]:
data = data.loc[data.plays != 0]

In [19]:
users = list(np.sort(data.user_id.unique()))
artists = list(np.sort(data.artist_id.unique()))
plays = list(data.plays)

In [20]:
users[:5]

[0, 1, 2, 3, 4]

In [21]:
artists[:5]

[0, 1, 2, 3, 4]

In [22]:
plays[:5]

[1099, 897, 717, 706, 691]

In [23]:
rows = data.user_id.astype(int)
cols = data.artist_id.astype(int)

In [27]:
data_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users), len(artists)))

In [31]:
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse)

100%|██████████| 15.0/15 [01:46<00:00,  6.54s/it]


In [33]:
userid = 0

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

In [35]:
data.join

,plays,user_id,artist_id
0,1099,0,90933
1,897,0,185367
2,717,0,106704
3,706,0,155241
4,691,0,220128


In [34]:
recommendations

[(285585, 0.00018143759),
 (105337, 0.00018134947),
 (194208, 0.00018036528),
 (227975, 0.00017925828),
 (72905, 0.00017579235),
 (346005, 0.00017354556),
 (619, 0.0001713883),
 (283253, 0.00017066987),
 (57093, 0.00017058977),
 (80749, 0.00016990374)]

In [ ]:
related = model.similar_items(itemid)

In [ ]:
related